<h1>Data Processing</h1>

<h3>Importpath</h3>

Add Projectfolder to Importpath

In [1]:
import sys
import os
cwd = os.path.abspath(".")
sys.path.insert(0, cwd.replace("Notebook", "Code"))

sys.path

['/home/avatar/Dokumente/Projekte/Python/EconomicCalendarImpactAnalysis/Code',
 '/home/avatar/Dokumente/Projekte/Python/EconomicCalendarImpactAnalysis/Notebook',
 '/home/avatar/.pyenv/versions/3.10.13/lib/python310.zip',
 '/home/avatar/.pyenv/versions/3.10.13/lib/python3.10',
 '/home/avatar/.pyenv/versions/3.10.13/lib/python3.10/lib-dynload',
 '',
 '/home/avatar/.local/lib/python3.10/site-packages',
 '/home/avatar/.pyenv/versions/3.10.13/lib/python3.10/site-packages']

<h3>Imports</h3>

In [ ]:
import pandas as pd

import fmp
import influx as db

<h3>Data Processing</h3>

In [ ]:
start_date = pd.Timestamp("2024-01-01")
stop_date = pd.Timestamp("2024-01-07")

FMP = fmp.FinancialModelingPrep()
FMP.economic_calendar_pipeline(start_date, stop_date)

In [ ]:
FMP = fmp.FinancialModelingPrep()
FMP.economic_calendar_pipeline_longrange()

<h3>Test new Configuration for InfluxDB</h3>

In [ ]:
import influx as db

db.InfluxDatabase()

<h3>Data Processing: Pricedata form MetaTrader5 CSV's</h3>

In [8]:
import influx
import mt5

db = influx.InfluxDatabase()
mt5 = mt5.MetaTrader5()

majors = ["AUDUSD", "EURUSD", "GBPUSD", "NZDUSD", "USDCAD", "USDCHF", "USDJPY"]

major = majors[6]

print("Step #1")
raw_data = mt5.load_csv_to_dataframe(file_name=f"MetaTrader5 {major} 1MIN Full Year 2023.csv")
print("Step #2")
clean_data = mt5.preprocess_csv_dataframe(raw_data)
print("Step #3")
db.ingest_data(clean_data)

clean_data


Step #1


Step #2
Step #3
Counter: 1, Start at: 0, Stop at: 5000
Counter: 2, Start at: 5000, Stop at: 10000
Counter: 3, Start at: 10000, Stop at: 15000
Counter: 4, Start at: 15000, Stop at: 20000
Counter: 5, Start at: 20000, Stop at: 25000
Counter: 6, Start at: 25000, Stop at: 30000
Counter: 7, Start at: 30000, Stop at: 35000
Counter: 8, Start at: 35000, Stop at: 40000
Counter: 9, Start at: 40000, Stop at: 45000
Counter: 10, Start at: 45000, Stop at: 50000
Counter: 11, Start at: 50000, Stop at: 55000
Counter: 12, Start at: 55000, Stop at: 60000
Counter: 13, Start at: 60000, Stop at: 65000
Counter: 14, Start at: 65000, Stop at: 70000
Counter: 15, Start at: 70000, Stop at: 75000
Counter: 16, Start at: 75000, Stop at: 80000
Counter: 17, Start at: 80000, Stop at: 85000
Counter: 18, Start at: 85000, Stop at: 90000
Counter: 19, Start at: 90000, Stop at: 95000
Counter: 20, Start at: 95000, Stop at: 100000
Counter: 21, Start at: 100000, Stop at: 105000
Counter: 22, Start at: 105000, Stop at: 110000
Coun

,open,high,low,close,spread,symbol,timeframe,source
timestamp,,,,,,,,
2023-01-02 07:01:00-02:00,0.92412,0.92426,0.92409,0.92426,2,EURUSD,1min,MetaTrader5
2023-01-02 07:02:00-02:00,0.92409,0.92410,0.92409,0.92409,13,EURUSD,1min,MetaTrader5
2023-01-02 07:03:00-02:00,0.92408,0.92408,0.92408,0.92408,13,EURUSD,1min,MetaTrader5
2023-01-02 07:04:00-02:00,0.92408,0.92410,0.92408,0.92410,13,EURUSD,1min,MetaTrader5
2023-01-02 07:05:00-02:00,0.92410,0.92411,0.92409,0.92409,13,EURUSD,1min,MetaTrader5
...,...,...,...,...,...,...,...,...
2023-12-29 23:54:00-02:00,0.84131,0.84131,0.84127,0.84128,19,EURUSD,1min,MetaTrader5
2023-12-29 23:55:00-02:00,0.84130,0.84130,0.84120,0.84123,10,EURUSD,1min,MetaTrader5
2023-12-29 23:56:00-02:00,0.84119,0.84163,0.84119,0.84139,19,EURUSD,1min,MetaTrader5
